# About

This notebook will introduce the `WITH` cypher keyword.

In [ ]:
from neo4j import GraphDatabase, Record, ResultSummary, EagerResult
from neo4j.time import Date

import pandas as pd
pd.set_option('display.max_colwidth', 100)

import os 
import sys
from dotenv import load_dotenv 
load_dotenv()

# Add the utils directory to sys.path
sys.path.append(os.path.abspath("../utils"))

from Neo4jParser import Neo4jParser


NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## `WITH`

* The `WITH` clause allows you to use specific variables in the successive line of code.
* `WITH` can be used to write subqueries.
* If I have a query:<br>
    `MATCH (p:Person)--(m:Movie)`<br>
    `WITH p`<br>
    `RETURN m.title`<br>
    * This query will throw an error because we specified "WITH p" which tells Neo4j, "After this line, only remember the variables and values I specified.
* Expressions can be used in `WITH` but they MUST be aliased.
* Variables can be ordered within the `WITH` clause.
* `WITH` allows you to run all `RETURN` commands like `SKIP`, `LIMIT`, `ORDER BY`, etc. These actions have to be done before returning an output in a list format using a functin like `collect()`.
* `WITH` can be powerfully used in confuction with `OPTIONAL MATCH`. Say you query a pattern using `MATCH` and you want to see if a node in your first pattern is matched with another pattern. `WITH` let's you handle this all in the same query.

In [2]:
# Demonstrate how with restricts variables
result = driver.execute_query(
    """ 
    MATCH (p:Person)--(m:Movie)
    WITH p 
    RETURN *
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
# Notice in data.keys() only one variable is returned 'p' since we delcared "WITH p"
data.keys()

Started streaming 255 records after 0 ms and completed after 2 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person)--(m:Movie)
    WITH p 
    RETURN *
    


dict_keys(['p'])

In [3]:
# Use with clause with an expression
result = driver.execute_query(
    """ 
    MATCH (p:Person)
    WITH toUpper(labels(p)[0]) AS upperLabels, p
    RETURN *
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)

# For each key returned, show the top 5 records
head = {}
for key in data.keys():
    head[key] = data[key][:5]

head

Started streaming 139 records after 1 ms and completed after 2 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person)
    WITH toUpper(labels(p)[0]) AS upperLabels, p
    RETURN *
    


{'upperLabels': ['PERSON', 'PERSON', 'PERSON', 'PERSON', 'PERSON'],
 'p': [{'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:1',
   'labels': frozenset({'Person'}),
   'properties': {'born': 1964, 'name': 'Keanu Reeves'}},
  {'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:2',
   'labels': frozenset({'Person'}),
   'properties': {'born': 1967, 'name': 'Carrie-Anne Moss'}},
  {'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:3',
   'labels': frozenset({'Person'}),
   'properties': {'born': 1961, 'name': 'Laurence Fishburne'}},
  {'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:4',
   'labels': frozenset({'Person'}),
   'properties': {'born': 1960, 'name': 'Hugo Weaving'}},
  {'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:5',
   'labels': frozenset({'Person'}),
   'properties': {'born': 1967, 'name': 'Andy Wachowski'}}]}

In [4]:
# Use with clause and order variables
result = driver.execute_query(
    """ 
    MATCH (p:Person)
    WITH p ORDER BY p.name
    RETURN * LIMIT 5
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)

# For each key returned, show the top 5 records
head = {}
for key in data.keys():
    head[key] = data[key][:5]

head

Started streaming 5 records after 1 ms and completed after 1 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person)
    WITH p ORDER BY p.name
    RETURN * LIMIT 5
    


{'p': [{'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:28',
   'labels': frozenset({'Person'}),
   'properties': {'born': 1961, 'name': 'Aaron Sorkin'}},
  {'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:13',
   'labels': frozenset({'Person'}),
   'properties': {'born': 1940, 'name': 'Al Pacino'}},
  {'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:5',
   'labels': frozenset({'Person'}),
   'properties': {'born': 1967, 'name': 'Andy Wachowski'}},
  {'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:166',
   'labels': frozenset({'Person'}),
   'properties': {'name': 'Angela Scope'}},
  {'elementId': '4:552b0252-2f83-4c7e-a0bf-f921a4b1b7cf:57',
   'labels': frozenset({'Person'}),
   'properties': {'born': 1960, 'name': 'Annabella Sciorra'}}]}

In [9]:
# Use skip, limit, order by with 'with' and return with 'collect()'
result = driver.execute_query(
    """ 
    MATCH (p:Person)
    WHERE p.born IS NOT NULL
    WITH p.born AS born SKIP 5 LIMIT 4 ORDER BY born DESC
    RETURN DISTINCT collect(born)
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 42 ms and completed after 43 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person)
    WHERE p.born IS NOT NULL
    WITH p.born AS born SKIP 5 LIMIT 4 ORDER BY born DESC
    RETURN DISTINCT collect(born)
    


{'collect(born)': [[1978, 1975, 1965, 1952]]}

In [14]:
# Use 'with' to show the movies the other movies that were acting in by the same actors who acted in "A Few Good Men"
result = driver.execute_query(
    """ 
    MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "A Few Good Men"})
    WITH p
    MATCH (p)-[:ACTED_IN]->(m:Movie)
    WHERE m.title <> "A Few Good Men" 
    WITH DISTINCT m.title AS movies ORDER BY movies
    RETURN collect(movies) AS movies_list
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 86 ms and completed after 87 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "A Few Good Men"})
    WITH p
    MATCH (p)-[:ACTED_IN]->(m:Movie)
    WHERE m.title <> "A Few Good Men" 
    WITH DISTINCT m.title AS movies ORDER BY movies
    RETURN collect(movies) AS movies_list
    


{'movies_list': [['Apollo 13',
   'As Good as It Gets',
   'Frost/Nixon',
   'Hoffa',
   'Jerry Maguire',
   "One Flew Over the Cuckoo's Nest",
   "Something's Gotta Give",
   'Stand By Me',
   'Top Gun',
   'What Dreams May Come']]}

In [20]:
# Show the director of the most recent movie "Meg Ryan" acted in
result = driver.execute_query(
    """ 
    MATCH (p:Person {name: "Meg Ryan"})-[:ACTED_IN]->(m:Movie)
    WITH m ORDER BY m.released DESC LIMIT 1 
    OPTIONAL MATCH (m)<-[:DIRECTED]-(p:Person)
    RETURN p.name
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 16 ms and completed after 17 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person {name: "Meg Ryan"})-[:ACTED_IN]->(m:Movie)
    WITH m ORDER BY m.released DESC LIMIT 1 
    OPTIONAL MATCH (m)<-[:DIRECTED]-(p:Person)
    RETURN p.name
    


{'p.name': ['Nora Ephron']}